In [1]:
!pip install streamlit google-generativeai gtts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 93.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.3 MB/s eta 0:00:00


In [18]:
%%writefile app.py
import streamlit as st
import os
import time
import numpy as np
import google.generativeai as genai
from gtts import gTTS
import re
import base64
from io import BytesIO
import json

if 'initialized' not in st.session_state:
    st.session_state.initialized = False
    st.session_state.story_started = False
    st.session_state.current_node_id = None
    st.session_state.nodes = {}
    st.session_state.history = []
    st.session_state.choice_history = []
    st.session_state.audio_cache = {}
    st.session_state.story_config = None

st.set_page_config(
    page_title="KukuVerse - Interactive Audio Stories",
    page_icon="📖",
    layout="centered",
    initial_sidebar_state="expanded"
)

st.markdown("""
<style>
    .story-title {
        font-size: 2.5rem;
        font-weight: bold;
        color: #1E88E5;
        margin-bottom: 1rem;
    }
    .story-content {
        font-size: 1.2rem;
        line-height: 1.6;
        margin-bottom: 1.5rem;
        padding: 1.5rem;
        background-color: #f0f5ff;
        border-radius: 10px;
        border-left: 5px solid #1E88E5;
    }
    .choice-button {
        margin-bottom: 10px;
    }
    .footer {
        margin-top: 50px;
        text-align: center;
        font-size: 0.8rem;
        color: #888;
    }
    .divider {
        height: 3px;
        background-color: #1E88E5;
        margin: 20px 0;
    }
</style>
""", unsafe_allow_html=True)

class StoryConfig:
    def __init__(self, title, premise, protagonist, setting, genre, max_choices_per_node=2):
        self.title = title
        self.premise = premise
        self.protagonist = protagonist
        self.setting = setting
        self.genre = genre
        self.max_choices_per_node = max_choices_per_node

class StoryNode:
    def __init__(self, node_id, content, choices=None):
        self.node_id = node_id
        self.content = content
        self.choices = choices if choices else []
        self.audio_data = None

class NarrativeEngine:
    def __init__(self, api_key):
        genai.configure(api_key=api_key)
        self.model = genai.GenerativeModel('gemini-2.0-flash')

    def generate_story_start(self, config):
        prompt = f"""
        Create an engaging beginning for an interactive audio story with the following details:
        Title: {config.title}
        Premise: {config.premise}
        Protagonist: {config.protagonist}
        Setting: {config.setting}
        Genre: {config.genre}

        Write an immersive first-person narrative (250-300 words) that sets up the story and ends with a situation
        where the protagonist (the listener) faces a choice. The narrative should be engaging and descriptive,
        making the listener feel like they're part of the story. Do not include any prompts for choices yet.
        """

        with st.spinner("Generating your story beginning..."):
            response = self.model.generate_content(prompt)
            return response.text

    def generate_choices(self, config, current_situation, history=None):
        history_context = ""
        if history and len(history) > 0:
            history_context = "Previous choices made: " + ", ".join(history)

        prompt = f"""
        Based on the following situation in an interactive story, generate exactly {config.max_choices_per_node} distinct and interesting choices
        for the protagonist to make next:

        Story title: {config.title}
        Genre: {config.genre}
        Current situation: {current_situation}
        {history_context}

        Format your response exactly like this example:
        1. [Brief description of first choice]
        2. [Brief description of second choice]

        Make each choice concise (10-15 words), distinct (leading to different story paths), and compelling.
        """

        with st.spinner("Considering your options..."):
            response = self.model.generate_content(prompt)
            choices_text = response.text.strip()

            choices = []
            for line in choices_text.split('\n'):
                if re.match(r'^\d+\.', line.strip()):
                    choice_text = re.sub(r'^\d+\.\s*', '', line.strip())
                    choices.append(choice_text)

            while len(choices) > config.max_choices_per_node:
                choices.pop()

            while len(choices) < config.max_choices_per_node:
                choices.append(f"Choice {len(choices)+1}")

            return choices

    def generate_next_segment(self, config, previous_content, selected_choice, node_id):
        prompt = f"""
        Continue this interactive story based on the user's choice:

        Story title: {config.title}
        Genre: {config.genre}
        Previous segment: {previous_content}

        User chose: {selected_choice}

        Write the next segment of the story (250-300 words) that follows from this choice.
        Continue in first-person narrative style that's immersive and descriptive.
        If this is node_id "{node_id}" and is deeper than 3 levels in the story, consider bringing
        the story toward a conclusion, but still end with a new situation requiring a choice.

        Do not include the choices themselves in your response, just the narrative.
        """

        with st.spinner("Continuing your story..."):
            response = self.model.generate_content(prompt)
            return response.text

    def generate_ending(self, config, previous_content, selected_choice):
        prompt = f"""
        Create a satisfying conclusion to this interactive story based on the final choice:

        Story title: {config.title}
        Genre: {config.genre}
        Previous segment: {previous_content}

        Final choice made: {selected_choice}

        Write a conclusive ending (250-300 words) that wraps up the story based on this choice.
        Keep the first-person narrative style and make the ending feel meaningful based on the
        choices that led here. This is the final part of the story, so provide closure.
        """

        with st.spinner("Creating your story's ending..."):
            response = self.model.generate_content(prompt)
            return response.text

class TTSEngine:
    def __init__(self):
        pass

    def generate_audio(self, text):
        mp3_fp = BytesIO()
        tts = gTTS(text=text, lang='en', slow=False)
        tts.write_to_fp(mp3_fp)
        mp3_fp.seek(0)
        audio_data = mp3_fp.read()
        return audio_data

    def get_audio_html(self, audio_data):
        audio_base64 = base64.b64encode(audio_data).decode()
        return f'<audio autoplay controls><source src="data:audio/mp3;base64,{audio_base64}" type="audio/mp3"></audio>'

class KukuVerse:
    def __init__(self, api_key, story_config):
        self.config = story_config
        self.narrative_engine = NarrativeEngine(api_key)
        self.tts_engine = TTSEngine()

    def initialize_story(self):
        start_content = self.narrative_engine.generate_story_start(self.config)

        start_choices = self.narrative_engine.generate_choices(
            self.config, start_content
        )
        start_node = StoryNode("start", start_content, start_choices)
        audio_data = self.tts_engine.generate_audio(start_content)
        start_node.audio_data = audio_data
        st.session_state.nodes["start"] = start_node.__dict__
        st.session_state.current_node_id = "start"
        st.session_state.story_started = True

        return start_node

    def make_choice(self, choice_index):
        current_node_dict = st.session_state.nodes.get(st.session_state.current_node_id)

        if not current_node_dict or choice_index >= len(current_node_dict["choices"]):
            st.error("Invalid choice!")
            return None

        selected_choice = current_node_dict["choices"][choice_index]
        st.session_state.choice_history.append(selected_choice)
        next_node_id = f"{st.session_state.current_node_id}_{choice_index}"
        if len(st.session_state.choice_history) >= 3 and np.random.random() > 0.5:
            ending_content = self.narrative_engine.generate_ending(
                self.config, current_node_dict["content"], selected_choice
            )
            ending_node = StoryNode(next_node_id, ending_content, [])
            ending_node.audio_data = self.tts_engine.generate_audio(ending_content)
            st.session_state.nodes[next_node_id] = ending_node.__dict__
            st.session_state.history.append(st.session_state.current_node_id)
            st.session_state.current_node_id = next_node_id

            return ending_node.__dict__
        else:
            next_content = self.narrative_engine.generate_next_segment(
                self.config, current_node_dict["content"], selected_choice, next_node_id
            )
            next_choices = self.narrative_engine.generate_choices(
                self.config, next_content, st.session_state.choice_history
            )
            next_node = StoryNode(next_node_id, next_content, next_choices)
            next_node.audio_data = self.tts_engine.generate_audio(next_content)
            st.session_state.nodes[next_node_id] = next_node.__dict__
            st.session_state.history.append(st.session_state.current_node_id)
            st.session_state.current_node_id = next_node_id

            return next_node.__dict__

def setup_page():
    st.markdown('<div class="story-title">KukuVerse - Interactive Audio Stories</div>', unsafe_allow_html=True)
    st.markdown('Create and experience AI-generated interactive audio stories with branching narratives.')

    with st.sidebar:
        st.markdown("## Story Settings")
        api_key = st.text_input("Google AI Studio API Key", type="password")
        title = st.text_input("Story Title", value="The Venture Gamble")
        genre = st.selectbox("Genre", ["Adventure", "Mystery", "Romance", "Fantasy", "Sci-Fi", "Horror", "Comedy"], index=0)
        protagonist = st.text_input("Protagonist Description", value="an ambitious tech entrepreneur who’s willing to risk everything for their startup dream")
        setting = st.text_input("Story Setting", value="the fast-paced, cutthroat world of Silicon Valley, where innovation meets manipulation")
        premise = st.text_area("Story Premise", value="After a failed pitch, you're offered a mysterious algorithm that claims to predict investor behavior. But each use of it changes the startup landscape in unpredictable ways, forcing you to decide between ethics and success.")

        max_choices = st.radio("Maximum Choices per Decision Point", [2, 3, 4], index=0)

        if st.button("Start New Story", use_container_width=True):
            if not api_key:
                st.sidebar.error("Please enter your Google AI Studio API Key")
                return False

            story_config = StoryConfig(
                title=title,
                premise=premise,
                protagonist=protagonist,
                setting=setting,
                genre=genre.lower(),
                max_choices_per_node=max_choices
            )
            st.session_state.story_started = False
            st.session_state.current_node_id = None
            st.session_state.nodes = {}
            st.session_state.history = []
            st.session_state.choice_history = []
            st.session_state.story_config = {
                "title": title,
                "premise": premise,
                "protagonist": protagonist,
                "setting": setting,
                "genre": genre.lower(),
                "max_choices_per_node": max_choices
            }
            st.session_state.api_key = api_key
            st.session_state.initialized = True

            return True
    return False

def present_current_node():
    current_node_dict = st.session_state.nodes.get(st.session_state.current_node_id)

    if not current_node_dict:
        st.error("Story node not found!")
        return

    config = st.session_state.story_config
    st.markdown(f'<div class="story-title">{config["title"]}</div>', unsafe_allow_html=True)
    formatted_content = current_node_dict["content"].replace("\n\n", "\n").strip()
    st.markdown(f'<div class="story-content">{formatted_content}</div>', unsafe_allow_html=True)
    if current_node_dict["audio_data"]:
        audio_html = TTSEngine().get_audio_html(current_node_dict["audio_data"])
        st.markdown(audio_html, unsafe_allow_html=True)

    st.markdown('<div class="divider"></div>', unsafe_allow_html=True)
    if current_node_dict["choices"] and len(current_node_dict["choices"]) > 0:
        st.markdown("### What will you do?")

        for i, choice in enumerate(current_node_dict["choices"]):
            choice_key = f"choice_{st.session_state.current_node_id}_{i}"
            if st.button(f"{i+1}. {choice}", key=choice_key, use_container_width=True):
                with st.spinner(f"You chose: {choice}"):
                    kukuverse = KukuVerse(
                        st.session_state.api_key,
                        StoryConfig(**st.session_state.story_config)
                    )
                    kukuverse.make_choice(i)
                    st.rerun()
    else:
        st.markdown("### THE END")
        if st.button("Start a New Story", use_container_width=True):
            st.session_state.story_started = False
            st.session_state.initialized = False
            st.rerun()

def main():
    new_story_requested = setup_page()
    if st.session_state.initialized:
        if new_story_requested or not st.session_state.story_started:
            kukuverse = KukuVerse(
                st.session_state.api_key,
                StoryConfig(**st.session_state.story_config)
            )
            kukuverse.initialize_story()
            st.rerun()
        else:
            present_current_node()

if __name__ == "__main__":
    main()

Writing app.py


External URL (example: 34.75.199.25) is password for streamlit app

URL for streamlit app (example: your url is: https://breezy-feet-buy.loca.lt)

In [19]:
!streamlit run app.py & npx localtunnel --port 8501



⠙⠹⠸
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.75.199.25:8501

⠼⠴your url is: https://breezy-feet-buy.loca.lt
  Stopping...
^C
